In [13]:
!pip install gradio google-generativeai newspaper3k bs4 requests lxml_html_clean

In [14]:
%%writefile app.py
import gradio as gr
import google.generativeai as genai
from newspaper import Article
from bs4 import BeautifulSoup
import requests

# Load API key
API_KEY = "YOUR_API_KEY"
genai.configure(api_key=API_KEY)

# Function to fetch article text
def fetch_article(url):
    """Extracts the main text from an article URL."""
    try:
        # Try newspaper3k
        article = Article(url)
        article.download()
        article.parse()
        article_text = article.text

        # Fallback: Use BeautifulSoup if newspaper3k fails
        if len(article_text) < 200:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            paragraphs = soup.find_all('p')
            article_text = " ".join([p.get_text() for p in paragraphs])

        return article_text.strip()
    except Exception as e:
        return f"Error fetching article: {str(e)}"

# Function to summarize text using Gemini 1.5 Flash
def summarize_with_gemini(url):
    """Fetches article text and summarizes it using Gemini AI."""
    article_text = fetch_article(url)
    if "Error" in article_text:
        return article_text

    try:
        model = genai.GenerativeModel("gemini-1.5-flash")
        prompt = f"Summarize the following article in third person while keeping all important concepts:\n\n{article_text}"
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"Error in summarization: {str(e)}"

# Gradio UI setup
gr_interface = gr.Interface(
    fn=summarize_with_gemini,
    inputs=gr.Textbox(label="Enter the article URL:"),
    outputs=gr.Textbox(label="AI-Generated Summary"),
    title="📰 AI Article Summarizer",
    description="Enter a URL of an article, and the system will fetch and summarize it using Gemini AI.",
    theme="default",
)

# Launch the app
if __name__ == "__main__":
    gr_interface.launch(share=True)


Overwriting app.py


In [16]:
!python app.py

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://91b22c186ebaa7be7e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2870, in block_thread
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/app.py", line 59, in <module>
    gr_interface.launch(share=True)
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2775, in launch
    self.block_thread()
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2874, in block_thread
    self.server.close()
  File "/usr/local/lib/python3.11/dis